## 명탐정 코난 매니아 판독기

### OpenAI LLM 준비 및 퀴즈 파일 지정
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import gradio as gr
import random, json, os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 퀴즈 파일 및 출제 문항 개수 지정
QUIZ_FILE = "conan_quiz.json"
QUIZ_COUNT = 3

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)


### 퀴즈 문항 (`conan_quiz.json`) 로딩
- `def load_quiz()` :	load_quiz라는 이름의 함수를 정의
- `with open(QUIZ_FILE, "r", encoding="utf-8") as f`: 지정된 경로의 JSON 파일(QUIZ_FILE)을 UTF-8 인코딩 방식으로 읽기 모드("r")로 열고, with 구문을 사용하여 파일 사용 후 자동으로 닫히도록 처리
- `all_q = json.load(f)` : JSON 파일 내용을 파싱하여 파이썬 객체(리스트 형태, 퀴즈 문제들)로 변환.
- `return random.sample(all_q, QUIZ_COUNT)` :전체 문제 중 QUIZ_COUNT 개수만큼 무작위로 샘플링하여 반환(중복 없이 선택)

In [ ]:
# 퀴즈 로딩 함수
def load_quiz():
    with open(QUIZ_FILE, "r", encoding="utf-8") as f:
        all_q = json.load(f)
    return random.sample(all_q, QUIZ_COUNT)

### 문제 출력 구성

In [ ]:
# 문제 출력 - 선다형 구성
def get_question(state):
    idx = state["quiz_index"]
    q = state["questions"][idx]
    text = f"문제 {idx+1}: {q['question']}"
    if q["type"] == "multiple_choice":
        choices = [f"{i+1}. {c}" for i, c in enumerate(q["choices"])]
        text += "\n" + "\n".join(choices)
    return text

### 사용자 응답 구성

In [ ]:
# 사용자 답변을 상태에 저장
def update_state(state, user_input):
    idx = state["quiz_index"]
    q = state["questions"][idx]
    processed = user_input.strip()

    if q["type"] == "multiple_choice":
        try:
            sel = int(processed) - 1
            if 0 <= sel < len(q["choices"]):
                processed = q["choices"][sel]
        except:
            pass

    state["user_answers"].append(
        {
            "question_text": q["question"],
            "user_response": processed,
            "is_correct": False,
            "correct_answer": str(q["answer"]),
        }
    )
    state["quiz_index"] += 1
    return state

### LLM으로 보낼 채점 프롬프트

In [ ]:
# 채점 프롬프트 생성
def build_grading_prompt(state):
    parts = [
        "당신은 퀴즈 채점관입니다. 사용자 답변을 정답 여부로 판단하고 각 문제에 피드백을 제공해주세요.",
        "마지막에는 '총점: X/Y' 형식으로 출력해주세요.",
    ]
    for i, (q, a) in enumerate(zip(state["questions"], state["user_answers"])):
        parts.append(f"\n문제 {i+1}: {q['question']}")
        if q["type"] == "multiple_choice":
            parts.append(f"선택지: {', '.join(q['choices'])}")
        parts.append(f"정답: {q['answer']}")
        parts.append(f"사용자 답변: {a['user_response']}")
    return "\n".join(parts)

### LCEL 채점 파이프라인 구성

In [ ]:
# LCEL 채점 체인
grade_chain = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "채점관으로서 정답 판단 및 피드백을 제공해주세요."),
            ("user", "{grading_input}"),
        ]
    )
    | llm
    | StrOutputParser()
)

### 퀴즈 시작 요청 처리

In [ ]:
# 퀴즈 시작 요청 처리
def handle_quiz_start(user_input, quiz_state, messages):
    quiz_state["questions"] = load_quiz()
    quiz_state["quiz_index"] = 0
    quiz_state["user_answers"] = []
    qtext = get_question(quiz_state)
    messages.append([user_input, qtext])
    return quiz_state, messages

### 퀴즈가 이미 끝난 경우

In [ ]:
# 퀴즈가 이미 끝난 경우
def handle_quiz_already_done(user_input, messages):
    messages.append(
        [
            user_input,
            "퀴즈가 이미 종료되었습니다. 다시 시작하려면 '퀴즈 시작'이라고 입력하세요.",
        ]
    )
    return messages

### 사용자 답변 처리

In [ ]:
# 사용자 답변 처리
def handle_user_answer(user_input, quiz_state, messages):
    quiz_state = update_state(quiz_state, user_input)

    if quiz_state["quiz_index"] < len(quiz_state["questions"]):
        qtext = get_question(quiz_state)
        messages.append([user_input, qtext])
    else:
        prompt = build_grading_prompt(quiz_state)
        result = grade_chain.invoke({"grading_input": prompt})
        messages.append([user_input, result])

    return quiz_state, messages

### 메인 챗봇 처리 함수

In [ ]:
# 메인 챗봇 처리 함수
def chat_fn(user_input, state):
    user_input_lower = user_input.strip().lower()
    messages = state["chat_history"]
    quiz_state = state["quiz_state"]

    if quiz_state["questions"] == []:
        if user_input_lower in ["퀴즈", "퀴즈 시작"]:
            quiz_state, messages = handle_quiz_start(user_input, quiz_state, messages)
        else:
            messages.append(
                [
                    user_input,
                    "'퀴즈' 또는 '퀴즈 시작'이라고 입력하면 퀴즈를 시작합니다.",
                ]
            )
    elif quiz_state["quiz_index"] >= len(quiz_state["questions"]):
        messages = handle_quiz_already_done(user_input, messages)
    else:
        quiz_state, messages = handle_user_answer(user_input, quiz_state, messages)

    state["quiz_state"] = quiz_state
    state["chat_history"] = messages
    return messages, state

### 초기 상태 정의

In [ ]:
# 상태 초기화
def init_state():
    return {
        "quiz_state": {"quiz_index": 0, "questions": [], "user_answers": []},
        "chat_history": [],
    }

## Gradio UI 정의

In [ ]:
# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🕵️ 명탐정 코난 매니아 판별기")
    chatbot = gr.Chatbot(label="명탐정 코난 퀴즈 챗봇", height=400)
    txt = gr.Textbox(placeholder="'퀴즈 시작'을 입력해보세요!", show_label=False)
    state = gr.State(init_state())

    txt.submit(chat_fn, inputs=[txt, state], outputs=[chatbot, state])
    txt.submit(lambda: "", None, txt)

demo.launch()

-----
** End of Documents **